<a href="https://colab.research.google.com/github/head1ton/006_Medical_Analysis/blob/main/notebook/Brain_MRI_NIfTI_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치
!pip install nibabel matplotlib

In [ ]:
# 샘플 NIfTI 생성 코드
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# 경로 설정
output_dir = Path("nifti_sample")
output_dir.mkdir(exist_ok=True)

In [ ]:
# 샘플 데이터 생성
def generate_sample_nifti(shape=(64, 64, 32), mask_ratio=0.2):
    img_data = np.random.rand(*shape).astype(np.float32)
    mask_data = (np.random.rand(*shape) < mask_ratio).astype(np.uint8)

    affine = np.eye(4)
    img_nii = nib.Nifti1Image(img_data, affine)
    mask_nii = nib.Nifti1Image(mask_data, affine)
    return img_nii, mask_nii

In [ ]:
# 파일 생성 및 저장
img_nii, mask_nii = generate_sample_nifti()
nib.save(img_nii, str(output_dir / "sample_brain.nii.gz"))
nib.save(mask_nii, str(output_dir / "sample_mask.nii.gz"))

In [ ]:
# 중간 slice 시각화
middle_slice = img_nii.get_fdata()[:, :, img_nii.shape[-1] // 2]
plt.imshow(middle_slice, cmap="gray")
plt.title("🧠 Sample Brain MRI (Middle Slice)")
plt.axis("off")
plt.show()

In [ ]:
## nifti_loader
import os
import torch
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class BrainMRIDataset(Dataset):
    def __init__(self, img_path, mask_path, transform=None):
        self.img_nii = nib.load(img_path).get_fdata()
        self.mask_nii = nib.load(mask_path).get_fdata()
        self.transform = transform

        assert self.img_nii.shape == self.mask_nii.shape, "이미지와 마스크 shape이 다릅니다."
        self.slices = self.img_nii.shape[2]  # z축 기준 슬라이스 수

    def __len__(self):
        return self.slices

    def __getitem__(self, idx):
        img_slice = self.img_nii[:, :, idx]
        mask_slice = self.mask_nii[:, :, idx]

        # 정규화
        img_slice = (img_slice - np.min(img_slice)) / (np.max(img_slice) - np.min(img_slice) + 1e-5)

        # 차원 확장 (1, H, W)
        img = torch.tensor(img_slice, dtype=torch.float32).unsqueeze(0)
        mask = torch.tensor(mask_slice, dtype=torch.float32).unsqueeze(0)

        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)

        return img, mask


In [ ]:
# 경로 설정
img_path = "nifti_sample/sample_brain.nii.gz"
mask_path = "nifti_sample/sample_mask.nii.gz"

# 데이터셋 정의
dataset = BrainMRIDataset(img_path, mask_path)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoader 구성
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
## UNet++ (Nested UNet)
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNetPlusPlus(nn.Module):
    def __init__(self, in_ch=1, out_ch=1, filters=[32, 64, 128, 256, 512]):
        super(UNetPlusPlus, self).__init__()
        self.conv0_0 = ConvBlock(in_ch, filters[0])
        self.conv1_0 = ConvBlock(filters[0], filters[1])
        self.conv2_0 = ConvBlock(filters[1], filters[2])
        self.conv3_0 = ConvBlock(filters[2], filters[3])
        self.conv4_0 = ConvBlock(filters[3], filters[4])

        self.up1_0 = nn.ConvTranspose2d(filters[1], filters[0], 2, stride=2)
        self.up2_0 = nn.ConvTranspose2d(filters[2], filters[1], 2, stride=2)
        self.up3_0 = nn.ConvTranspose2d(filters[3], filters[2], 2, stride=2)
        self.up4_0 = nn.ConvTranspose2d(filters[4], filters[3], 2, stride=2)

        self.conv0_1 = ConvBlock(filters[0]+filters[0], filters[0])
        self.conv1_1 = ConvBlock(filters[1]+filters[1], filters[1])
        self.conv2_1 = ConvBlock(filters[2]+filters[2], filters[2])
        self.conv3_1 = ConvBlock(filters[3]+filters[3], filters[3])

        self.up1_1 = nn.ConvTranspose2d(filters[1], filters[0], 2, stride=2)
        self.up2_1 = nn.ConvTranspose2d(filters[2], filters[1], 2, stride=2)
        self.up3_1 = nn.ConvTranspose2d(filters[3], filters[2], 2, stride=2)

        self.conv0_2 = ConvBlock(filters[0]*3, filters[0])
        self.final = nn.Conv2d(filters[0], out_ch, 1)

    def forward(self, x):
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(F.max_pool2d(x0_0, 2))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up1_0(x1_0)], 1))

        x2_0 = self.conv2_0(F.max_pool2d(x1_0, 2))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up2_0(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up1_1(x1_1)], 1))

        x3_0 = self.conv3_0(F.max_pool2d(x2_0, 2))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up3_0(x3_0)], 1))

        out = self.final(x0_2)
        return torch.sigmoid(out)


In [ ]:
model = UNetPlusPlus(in_ch=1, out_ch=1)
x = torch.randn(1, 1, 64, 64)  # batch, channel, H, W
y = model(x)
print("Output shape:", y.shape)

In [ ]:
## loss_metric
import torch
import torch.nn as nn

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, pred, target):
        pred = pred.contiguous()
        target = target.contiguous()

        intersection = (pred * target).sum(dim=(2,3))
        dice = (2. * intersection + self.smooth) / (pred.sum(dim=(2,3)) + target.sum(dim=(2,3)) + self.smooth)
        return 1 - dice.mean()

def dice_score(pred, target, threshold=0.5):
    pred = (pred > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    return (2. * intersection / (union + 1e-5)).item()

In [ ]:
## train
import torch
from loss_metric import DiceLoss, dice_score
from tqdm import tqdm
import os

def train_model(model, train_loader, val_loader, device, epochs=20, lr=1e-3, save_path="unetpp_model.pt"):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = DiceLoss()
    bce = torch.nn.BCELoss()

    model.to(device)
    best_val_dice = 0.0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for img, mask in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            img, mask = img.to(device), mask.to(device)

            pred = model(img)
            loss = criterion(pred, mask) + bce(pred, mask)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_dice = 0
        with torch.no_grad():
            for img, mask in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                img, mask = img.to(device), mask.to(device)
                pred = model(img)
                val_dice += dice_score(pred, mask)

        avg_val_dice = val_dice / len(val_loader)
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | Val Dice: {avg_val_dice:.4f}")

        if avg_val_dice > best_val_dice:
            best_val_dice = avg_val_dice
            torch.save(model.state_dict(), save_path)
            print(f"✅ Best model saved to {save_path}")

def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    return model


In [ ]:
from unetpp import UNetPlusPlus
from train import train_model
from nifti_loader import BrainMRIDataset
from torch.utils.data import DataLoader, random_split
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNetPlusPlus(in_ch=1, out_ch=1)

dataset = BrainMRIDataset("nifti_sample/sample_brain.nii.gz", "nifti_sample/sample_mask.nii.gz")
train_ds, val_ds = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False)

train_model(model, train_loader, val_loader, device, epochs=10, save_path="unetpp_brain_mri.pt")


In [ ]:
## 예측 + 시각화 (NIfTI + UNet++)
import nibabel as nib
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
from unetpp import UNetPlusPlus
from train import load_model

def normalize(img):
    img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-5)
    return img

def visualize_prediction(model_path, image_path, mask_path=None, slice_idx=80):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load image
    img_nii = nib.load(image_path)
    img_data = img_nii.get_fdata()
    img = img_data[:, :, slice_idx]
    img = normalize(img)

    # Prepare input
    input_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    # Load model
    model = UNetPlusPlus(in_ch=1, out_ch=1)
    model = load_model(model, model_path, device)

    # Predict
    with torch.no_grad():
        output = model(input_tensor)
        pred = output.squeeze().cpu().numpy()
        pred_mask = (pred > 0.5).astype(np.uint8)

    # Optional: Load GT mask
    gt_mask = None
    if mask_path:
        mask_nii = nib.load(mask_path)
        gt_data = mask_nii.get_fdata()
        gt_mask = gt_data[:, :, slice_idx]

    # Visualization
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title("Original MRI")

    plt.subplot(1, 3, 2)
    if gt_mask is not None:
        plt.imshow(img, cmap='gray')
        plt.imshow(gt_mask, alpha=0.4, cmap='Reds')
        plt.title("Ground Truth Mask")
    else:
        plt.axis('off')

    plt.subplot(1, 3, 3)
    overlay = cv2.addWeighted((img * 255).astype(np.uint8), 1.0, (pred_mask * 255).astype(np.uint8), 0.5, 0)
    plt.imshow(overlay, cmap='gray')
    plt.title("Predicted Mask")

    plt.tight_layout()
    plt.show()


In [ ]:
visualize_prediction(
    model_path="unetpp_brain_mri.pt",
    image_path="nifti_sample/sample_brain.nii.gz",
    mask_path="nifti_sample/sample_mask.nii.gz",  # 생략 가능
    slice_idx=80
)

In [ ]:
## streamlit (streamlit run streamlit_app.py)
import streamlit as st
import nibabel as nib
import torch
import numpy as np
import cv2
import tempfile
import matplotlib.pyplot as plt
from unetpp import UNetPlusPlus
from train import load_model

st.set_page_config(layout="wide", page_title="Brain MRI Segmentation Demo")

def normalize(img):
    img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-5)
    return img

def load_nifti(file):
    nii = nib.load(file)
    return nii.get_fdata()

def predict_slice(model, image_slice, device):
    img = normalize(image_slice)
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model(tensor)
        pred = (pred.squeeze().cpu().numpy() > 0.5).astype(np.uint8)
    return pred

def show_overlay(image, pred_mask, title="Prediction Overlay"):
    overlay = cv2.addWeighted((image*255).astype(np.uint8), 1.0, (pred_mask*255).astype(np.uint8), 0.5, 0)
    st.image(overlay, caption=title, use_column_width=True, channels="GRAY")

# Sidebar
st.sidebar.title("Upload Files")
image_file = st.sidebar.file_uploader("Upload Brain MRI (.nii.gz)", type=["nii", "nii.gz"])
mask_file = st.sidebar.file_uploader("Upload Ground Truth Mask (Optional)", type=["nii", "nii.gz"])
model_file = st.sidebar.file_uploader("Upload Trained Model (.pt)", type=["pt"])
slice_idx = st.sidebar.slider("Slice Index", 0, 200, 80)

# Main
st.title("🧠 Brain MRI Segmentation with UNet++")

if image_file and model_file:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Save temp files
    with tempfile.NamedTemporaryFile(delete=False) as tmp_img:
        tmp_img.write(image_file.read())
        image_path = tmp_img.name

    with tempfile.NamedTemporaryFile(delete=False) as tmp_model:
        tmp_model.write(model_file.read())
        model_path = tmp_model.name

    # Load data
    img_data = load_nifti(image_path)
    img_slice = img_data[:, :, slice_idx]
    norm_img = normalize(img_slice)

    # Load model
    model = UNetPlusPlus(in_ch=1, out_ch=1)
    model = load_model(model, model_path, device)

    # Predict
    pred_mask = predict_slice(model, img_slice, device)

    # Display
    col1, col2, col3 = st.columns(3)

    with col1:
        st.image(norm_img, caption="Original MRI", use_column_width=True, channels="GRAY")
    with col2:
        if mask_file:
            with tempfile.NamedTemporaryFile(delete=False) as tmp_mask:
                tmp_mask.write(mask_file.read())
                mask_path = tmp_mask.name
            gt_mask = load_nifti(mask_path)[:, :, slice_idx]
            st.image(gt_mask, caption="Ground Truth Mask", use_column_width=True)
        else:
            st.info("No ground truth uploaded.")
    with col3:
        show_overlay(norm_img, pred_mask)

else:
    st.info("Please upload both MRI and model files.")

In [ ]:
### IR 논문 포멧 자동 리포트(LaTeX 포함)
# ir_report_generator.py (실행- python ir_report_generator.py)

import os
from datetime import datetime

TEMPLATE = r"""
\documentclass{article}
\usepackage{graphicx}
\usepackage{geometry}
\geometry{margin=1in}
\title{{\bfseries Brain MRI Segmentation Report}}
\author{{AI Medical Lab}}
\date{{\today}}

\begin{document}

\maketitle

\section*{1. Introduction}
This report summarizes the deep learning-based brain MRI segmentation task using UNet++ architecture.

\section*{2. Dataset}
The dataset used for training and evaluation consists of NIfTI-format brain MRI scans with corresponding ground truth segmentation masks. Preprocessing involved slice extraction and normalization.

\section*{3. Model Architecture}
The model employed is UNet++, a nested U-Net architecture known for improved segmentation performance in biomedical imaging.

\section*{4. Training Setup}

\begin{itemize}
  \item Epochs: {epochs}
  \item Batch Size: {batch_size}
  \item Optimizer: Adam
  \item Learning Rate: {learning_rate}
  \item Loss Function: BCE + Dice Loss
\end{itemize}

\section*{5. Evaluation Results}

\begin{itemize}
  \item Dice Score: {dice_score:.4f}
  \item IoU Score: {iou_score:.4f}
\end{itemize}

\section*{6. Visual Results}
\begin{figure}[h!]
\centering
\includegraphics[width=0.6\textwidth]{{{image_path}}}
\caption{Prediction result for slice {slice_idx} with mask overlay.}
\end{figure}

\end{document}
"""

def generate_latex_report(save_path, metrics, image_path, hyperparams):
    tex_code = TEMPLATE.format(
        dice_score=metrics['dice'],
        iou_score=metrics['iou'],
        image_path=image_path,
        slice_idx=hyperparams['slice_idx'],
        epochs=hyperparams['epochs'],
        batch_size=hyperparams['batch_size'],
        learning_rate=hyperparams['learning_rate']
    )

    tex_file = os.path.join(save_path, "segmentation_report.tex")
    with open(tex_file, "w") as f:
        f.write(tex_code)
    print(f"LaTeX report generated at {tex_file}")

# Example usage
if __name__ == "__main__":
    metrics = {'dice': 0.8721, 'iou': 0.7943}
    image_path = "overlay_result.png"
    hyperparams = {'slice_idx': 80, 'epochs': 50, 'batch_size': 8, 'learning_rate': 0.0001}
    generate_latex_report("./", metrics, image_path, hyperparams)


In [ ]:
## JSRT / Montgomery 데이터 기반 흉부 X-ray (DICOM) 확장

# dicom_xray_pipeline.py

import os
import glob
import pydicom
import numpy as np
import cv2
from torch.utils.data import Dataset

class DICOMXrayDataset(Dataset):
    def __init__(self, dicom_dir, mask_dir=None, transform=None):
        self.dicom_paths = sorted(glob.glob(os.path.join(dicom_dir, "*.dcm")))
        self.mask_paths = sorted(glob.glob(os.path.join(mask_dir, "*.png"))) if mask_dir else None
        self.transform = transform

    def __len__(self):
        return len(self.dicom_paths)

    def __getitem__(self, idx):
        dicom_path = self.dicom_paths[idx]
        ds = pydicom.dcmread(dicom_path)
        img = ds.pixel_array.astype(np.float32)
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-5)
        img = np.expand_dims(img, axis=0)

        sample = {"image": img}

        if self.mask_paths:
            mask_path = self.mask_paths[idx]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = (mask > 127).astype(np.float32)
            mask = np.expand_dims(mask, axis=0)
            sample["mask"] = mask

        if self.transform:
            sample = self.transform(sample)

        return sample

# --- Example usage ---
if __name__ == "__main__":
    dataset = DICOMXrayDataset(dicom_dir="/montgomery/images", mask_dir="/masks")
    print("Sample shape:", dataset[0]['image'].shape)


In [ ]:
## 뇌 MRI 세그멘테이션을 위한 NIfTI (.nii.gz) 버전

# dicom_xray_pipeline.py

import os
import glob
import pydicom
import numpy as np
import cv2
from torch.utils.data import Dataset

class DICOMXrayDataset(Dataset):
    def __init__(self, dicom_dir, mask_dir=None, transform=None):
        self.dicom_paths = sorted(glob.glob(os.path.join(dicom_dir, "*.dcm")))
        self.mask_paths = sorted(glob.glob(os.path.join(mask_dir, "*.png"))) if mask_dir else None
        self.transform = transform

    def __len__(self):
        return len(self.dicom_paths)

    def __getitem__(self, idx):
        dicom_path = self.dicom_paths[idx]
        ds = pydicom.dcmread(dicom_path)
        img = ds.pixel_array.astype(np.float32)
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-5)
        img = np.expand_dims(img, axis=0)

        sample = {"image": img}

        if self.mask_paths:
            mask_path = self.mask_paths[idx]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = (mask > 127).astype(np.float32)
            mask = np.expand_dims(mask, axis=0)
            sample["mask"] = mask

        if self.transform:
            sample = self.transform(sample)

        return sample

# --- Example usage ---
if __name__ == "__main__":
    dataset = DICOMXrayDataset(dicom_dir="/montgomery/images", mask_dir="/masks")
    print("Sample shape:", dataset[0]['image'].shape)


In [ ]:
## Nifti Brain Dataset

# nifti_brain_dataset.py

import os
import glob
import nibabel as nib
import numpy as np
from torch.utils.data import Dataset

class NiftiBrainMRIDataset(Dataset):
    def __init__(self, image_dir, mask_dir=None, transform=None):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, '*.nii*')))
        self.mask_paths = sorted(glob.glob(os.path.join(mask_dir, '*.nii*'))) if mask_dir else None
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img_nii = nib.load(img_path)
        img = img_nii.get_fdata().astype(np.float32)

        # Normalize image
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-5)

        # Select middle slice for 2D segmentation (can be extended to 3D)
        z = img.shape[2] // 2
        img_slice = img[:, :, z]
        img_slice = np.expand_dims(img_slice, axis=0)  # shape (1, H, W)

        sample = {"image": img_slice}

        if self.mask_paths:
            mask_path = self.mask_paths[idx]
            mask_nii = nib.load(mask_path)
            mask = mask_nii.get_fdata()
            mask_slice = (mask[:, :, z] > 0).astype(np.float32)
            mask_slice = np.expand_dims(mask_slice, axis=0)  # shape (1, H, W)
            sample["mask"] = mask_slice

        if self.transform:
            sample = self.transform(sample)

        return sample

# Example
if __name__ == '__main__':
    dataset = NiftiBrainMRIDataset(
        image_dir="/brain/images",
        mask_dir="/brain/masks"
    )
    print("Sample shape:", dataset[0]['image'].shape)


In [ ]:
## UNet++ 모델
# app.py (Gradio HuggingFace Spaces + HF Model Hub 다운로드 지원)

import gradio as gr
import numpy as np
import pydicom
import nibabel as nib
import torch
from torchvision import transforms
from unet_model import UNet
from unetpp_model import UNetPP
from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from huggingface_hub import hf_hub_download


# Auto-download model weights from Hugging Face Hub
def load_model_weights_local_or_hf(model, filename, repo_id, subfolder=None):
    if os.path.exists(filename):
        model.load_state_dict(torch.load(filename, map_location="cpu"))
    else:
        print(f"🔁 Downloading {filename} from HF Hub...")
        model_path = hf_hub_download(repo_id=repo_id, filename=os.path.basename(filename), subfolder=subfolder)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

# Load models
unet_model = UNet()
unet_model = load_model_weights_local_or_hf(
    unet_model,
    filename="results/xray_unet_epoch30.pth",
    repo_id="your-hf-username/xray-unet",
    subfolder="models"
)

unetpp_model = UNetPP()
unetpp_model = load_model_weights_local_or_hf(
    unetpp_model,
    filename="results/unetpp_epoch25.pth",
    repo_id="your-hf-username/mri-unetpp",
    subfolder="models"
)


# Utils
def preprocess(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    return transform(img).unsqueeze(0)

def overlay_prediction(img, mask):
    img = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img]*3, axis=-1)
    mask_bin = (mask > 0.5).astype(np.uint8)
    mask_rgb = np.zeros_like(img_rgb)
    mask_rgb[:, :, 1] = mask_bin * 255
    overlaid = (0.7 * img_rgb + 0.3 * mask_rgb).astype(np.uint8)
    return Image.fromarray(overlaid)


def predict_dcm(file):
    dcm = pydicom.dcmread(file.name)
    img = dcm.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    input_tensor = preprocess(img)
    with torch.no_grad():
        pred = torch.sigmoid(unet_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(img, pred)
    return result


def predict_nii(file, slice_index):
    nifti = nib.load(file.name)
    data = nifti.get_fdata()
    slice_img = data[:, :, slice_index]
    norm_img = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
    input_tensor = preprocess(norm_img)
    with torch.no_grad():
        pred = torch.sigmoid(unetpp_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(norm_img, pred)
    return result


def xray_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🩻 Chest X-ray Segmentation")
        file_input = gr.File(label="Upload DICOM (.dcm)")
        output_img = gr.Image(label="Overlayed Prediction")
        file_input.change(predict_dcm, inputs=file_input, outputs=output_img)
    return demo


def mri_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Brain MRI Segmentation")
        file_input = gr.File(label="Upload NIfTI (.nii.gz)")
        slice_slider = gr.Slider(0, 150, step=1, label="Slice Index", value=60)
        output_img = gr.Image(label="Overlayed Prediction")
        gr.Button("Predict").click(predict_nii, inputs=[file_input, slice_slider], outputs=output_img)
    return demo


def main():
    with gr.TabbedInterface([xray_interface(), mri_interface()], ["X-ray", "MRI"]):
        pass


if __name__ == '__main__':
    main()

In [ ]:
import gradio as gr
import numpy as np
import pydicom
import nibabel as nib
import torch
from torchvision import transforms
from unet_model import UNet
from unetpp_model import UNetPP
from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from huggingface_hub import hf_hub_download


# Auto-download model weights from Hugging Face Hub
def load_model_weights_local_or_hf(model, filename, repo_id, subfolder=None):
    if os.path.exists(filename):
        model.load_state_dict(torch.load(filename, map_location="cpu"))
    else:
        print(f"🔁 Downloading {filename} from HF Hub...")
        model_path = hf_hub_download(repo_id=repo_id, filename=os.path.basename(filename), subfolder=subfolder)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

# Load models
unet_model = UNet()
unet_model = load_model_weights_local_or_hf(
    unet_model,
    filename="results/xray_unet_epoch30.pth",
    repo_id="your-hf-username/xray-unet",
    subfolder="models"
)

unetpp_model = UNetPP()
unetpp_model = load_model_weights_local_or_hf(
    unetpp_model,
    filename="results/unetpp_epoch25.pth",
    repo_id="your-hf-username/mri-unetpp",
    subfolder="models"
)


# Utils
def preprocess(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    return transform(img).unsqueeze(0)

def overlay_prediction(img, mask):
    img = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img]*3, axis=-1)
    mask_bin = (mask > 0.5).astype(np.uint8)
    mask_rgb = np.zeros_like(img_rgb)
    mask_rgb[:, :, 1] = mask_bin * 255
    overlaid = (0.7 * img_rgb + 0.3 * mask_rgb).astype(np.uint8)
    return Image.fromarray(overlaid)


def predict_dcm(file):
    dcm = pydicom.dcmread(file.name)
    img = dcm.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    input_tensor = preprocess(img)
    with torch.no_grad():
        pred = torch.sigmoid(unet_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(img, pred)
    return result


def predict_nii(file, slice_index):
    nifti = nib.load(file.name)
    data = nifti.get_fdata()
    slice_img = data[:, :, slice_index]
    norm_img = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
    input_tensor = preprocess(norm_img)
    with torch.no_grad():
        pred = torch.sigmoid(unetpp_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(norm_img, pred)
    return result


def xray_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🩻 Chest X-ray Segmentation")
        file_input = gr.File(label="Upload DICOM (.dcm)")
        output_img = gr.Image(label="Overlayed Prediction")
        file_input.change(predict_dcm, inputs=file_input, outputs=output_img)
    return demo


def mri_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Brain MRI Segmentation")
        file_input = gr.File(label="Upload NIfTI (.nii.gz)")
        slice_slider = gr.Slider(0, 150, step=1, label="Slice Index", value=60)
        output_img = gr.Image(label="Overlayed Prediction")
        gr.Button("Predict").click(predict_nii, inputs=[file_input, slice_slider], outputs=output_img)
    return demo


def main():
    with gr.TabbedInterface([xray_interface(), mri_interface()], ["X-ray", "MRI"]):
        pass


if __name__ == '__main__':
    main()

In [ ]:
# streamlit run inference_demo.py

from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from huggingface_hub import hf_hub_download


# Auto-download model weights from Hugging Face Hub
def load_model_weights_local_or_hf(model, filename, repo_id, subfolder=None):
    if os.path.exists(filename):
        model.load_state_dict(torch.load(filename, map_location="cpu"))
    else:
        print(f"🔁 Downloading {filename} from HF Hub...")
        model_path = hf_hub_download(repo_id=repo_id, filename=os.path.basename(filename), subfolder=subfolder)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

# Load models
unet_model = UNet()
unet_model = load_model_weights_local_or_hf(
    unet_model,
    filename="results/xray_unet_epoch30.pth",
    repo_id="your-hf-username/xray-unet",
    subfolder="models"
)

unetpp_model = UNetPP()
unetpp_model = load_model_weights_local_or_hf(
    unetpp_model,
    filename="results/unetpp_epoch25.pth",
    repo_id="your-hf-username/mri-unetpp",
    subfolder="models"
)


# Utils
def preprocess(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    return transform(img).unsqueeze(0)

def overlay_prediction(img, mask):
    img = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img]*3, axis=-1)
    mask_bin = (mask > 0.5).astype(np.uint8)
    mask_rgb = np.zeros_like(img_rgb)
    mask_rgb[:, :, 1] = mask_bin * 255
    overlaid = (0.7 * img_rgb + 0.3 * mask_rgb).astype(np.uint8)
    return Image.fromarray(overlaid)


def predict_dcm(file):
    dcm = pydicom.dcmread(file.name)
    img = dcm.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    input_tensor = preprocess(img)
    with torch.no_grad():
        pred = torch.sigmoid(unet_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(img, pred)
    return result


def predict_nii(file, slice_index):
    nifti = nib.load(file.name)
    data = nifti.get_fdata()
    slice_img = data[:, :, slice_index]
    norm_img = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
    input_tensor = preprocess(norm_img)
    with torch.no_grad():
        pred = torch.sigmoid(unetpp_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(norm_img, pred)
    return result


def xray_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🩻 Chest X-ray Segmentation")
        file_input = gr.File(label="Upload DICOM (.dcm)")
        output_img = gr.Image(label="Overlayed Prediction")
        file_input.change(predict_dcm, inputs=file_input, outputs=output_img)
    return demo


def mri_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Brain MRI Segmentation")
        file_input = gr.File(label="Upload NIfTI (.nii.gz)")
        slice_slider = gr.Slider(0, 150, step=1, label="Slice Index", value=60)
        output_img = gr.Image(label="Overlayed Prediction")
        gr.Button("Predict").click(predict_nii, inputs=[file_input, slice_slider], outputs=output_img)
    return demo


def main():
    with gr.TabbedInterface([xray_interface(), mri_interface()], ["X-ray", "MRI"]):
        pass


if __name__ == '__main__':
    main()


In [ ]:
# app.py (Gradio HuggingFace Spaces + HF Model Hub 다운로드 지원)

import gradio as gr
import numpy as np
import pydicom
import nibabel as nib
import torch
from torchvision import transforms
from unet_model import UNet
from unetpp_model import UNetPP
from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from huggingface_hub import hf_hub_download


# Auto-download model weights from Hugging Face Hub
def load_model_weights_local_or_hf(model, filename, repo_id, subfolder=None):
    if os.path.exists(filename):
        model.load_state_dict(torch.load(filename, map_location="cpu"))
    else:
        print(f"🔁 Downloading {filename} from HF Hub...")
        model_path = hf_hub_download(repo_id=repo_id, filename=os.path.basename(filename), subfolder=subfolder)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

# Load models
unet_model = UNet()
unet_model = load_model_weights_local_or_hf(
    unet_model,
    filename="results/xray_unet_epoch30.pth",
    repo_id="your-hf-username/xray-unet",
    subfolder="models"
)

unetpp_model = UNetPP()
unetpp_model = load_model_weights_local_or_hf(
    unetpp_model,
    filename="results/unetpp_epoch25.pth",
    repo_id="your-hf-username/mri-unetpp",
    subfolder="models"
)


# Utils
def preprocess(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    return transform(img).unsqueeze(0)

def overlay_prediction(img, mask):
    img = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img]*3, axis=-1)
    mask_bin = (mask > 0.5).astype(np.uint8)
    mask_rgb = np.zeros_like(img_rgb)
    mask_rgb[:, :, 1] = mask_bin * 255
    overlaid = (0.7 * img_rgb + 0.3 * mask_rgb).astype(np.uint8)
    return Image.fromarray(overlaid)


def predict_dcm(file):
    dcm = pydicom.dcmread(file.name)
    img = dcm.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    input_tensor = preprocess(img)
    with torch.no_grad():
        pred = torch.sigmoid(unet_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(img, pred)
    return result


def predict_nii(file, slice_index):
    nifti = nib.load(file.name)
    data = nifti.get_fdata()
    slice_img = data[:, :, slice_index]
    norm_img = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
    input_tensor = preprocess(norm_img)
    with torch.no_grad():
        pred = torch.sigmoid(unetpp_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(norm_img, pred)
    return result


def xray_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🩻 Chest X-ray Segmentation")
        file_input = gr.File(label="Upload DICOM (.dcm)")
        output_img = gr.Image(label="Overlayed Prediction")
        file_input.change(predict_dcm, inputs=file_input, outputs=output_img)
    return demo


def mri_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Brain MRI Segmentation")
        file_input = gr.File(label="Upload NIfTI (.nii.gz)")
        slice_slider = gr.Slider(0, 150, step=1, label="Slice Index", value=60)
        output_img = gr.Image(label="Overlayed Prediction")
        gr.Button("Predict").click(predict_nii, inputs=[file_input, slice_slider], outputs=output_img)
    return demo


def main():
    with gr.TabbedInterface([xray_interface(), mri_interface()], ["X-ray", "MRI"]):
        pass


if __name__ == '__main__':
    main()


In [ ]:
## Chest X-ray DICOM 실시간 예측 데모 (Streamlit)

# app.py (Gradio HuggingFace Spaces + HF Model Hub 다운로드 지원)
## streamlit run dicom_xray_streamlit.py

import gradio as gr
import numpy as np
import pydicom
import nibabel as nib
import torch
from torchvision import transforms
from unet_model import UNet
from unetpp_model import UNetPP
from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from huggingface_hub import hf_hub_download


# Auto-download model weights from Hugging Face Hub
def load_model_weights_local_or_hf(model, filename, repo_id, subfolder=None):
    if os.path.exists(filename):
        model.load_state_dict(torch.load(filename, map_location="cpu"))
    else:
        print(f"🔁 Downloading {filename} from HF Hub...")
        model_path = hf_hub_download(repo_id=repo_id, filename=os.path.basename(filename), subfolder=subfolder)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

# Load models
unet_model = UNet()
unet_model = load_model_weights_local_or_hf(
    unet_model,
    filename="results/xray_unet_epoch30.pth",
    repo_id="your-hf-username/xray-unet",
    subfolder="models"
)

unetpp_model = UNetPP()
unetpp_model = load_model_weights_local_or_hf(
    unetpp_model,
    filename="results/unetpp_epoch25.pth",
    repo_id="your-hf-username/mri-unetpp",
    subfolder="models"
)


# Utils
def preprocess(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    return transform(img).unsqueeze(0)

def overlay_prediction(img, mask):
    img = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img]*3, axis=-1)
    mask_bin = (mask > 0.5).astype(np.uint8)
    mask_rgb = np.zeros_like(img_rgb)
    mask_rgb[:, :, 1] = mask_bin * 255
    overlaid = (0.7 * img_rgb + 0.3 * mask_rgb).astype(np.uint8)
    return Image.fromarray(overlaid)


def predict_dcm(file):
    dcm = pydicom.dcmread(file.name)
    img = dcm.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    input_tensor = preprocess(img)
    with torch.no_grad():
        pred = torch.sigmoid(unet_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(img, pred)
    return result


def predict_nii(file, slice_index):
    nifti = nib.load(file.name)
    data = nifti.get_fdata()
    slice_img = data[:, :, slice_index]
    norm_img = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
    input_tensor = preprocess(norm_img)
    with torch.no_grad():
        pred = torch.sigmoid(unetpp_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(norm_img, pred)
    return result


def xray_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🩻 Chest X-ray Segmentation")
        file_input = gr.File(label="Upload DICOM (.dcm)")
        output_img = gr.Image(label="Overlayed Prediction")
        file_input.change(predict_dcm, inputs=file_input, outputs=output_img)
    return demo


def mri_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Brain MRI Segmentation")
        file_input = gr.File(label="Upload NIfTI (.nii.gz)")
        slice_slider = gr.Slider(0, 150, step=1, label="Slice Index", value=60)
        output_img = gr.Image(label="Overlayed Prediction")
        gr.Button("Predict").click(predict_nii, inputs=[file_input, slice_slider], outputs=output_img)
    return demo


def main():
    with gr.TabbedInterface([xray_interface(), mri_interface()], ["X-ray", "MRI"]):
        pass


if __name__ == '__main__':
    main()


In [ ]:
### Flask API 서버

# app.py (Gradio HuggingFace Spaces + HF Model Hub 다운로드 지원)

import gradio as gr
import numpy as np
import pydicom
import nibabel as nib
import torch
from torchvision import transforms
from unet_model import UNet
from unetpp_model import UNetPP
from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from huggingface_hub import hf_hub_download


# Auto-download model weights from Hugging Face Hub
def load_model_weights_local_or_hf(model, filename, repo_id, subfolder=None):
    if os.path.exists(filename):
        model.load_state_dict(torch.load(filename, map_location="cpu"))
    else:
        print(f"🔁 Downloading {filename} from HF Hub...")
        model_path = hf_hub_download(repo_id=repo_id, filename=os.path.basename(filename), subfolder=subfolder)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

# Load models
unet_model = UNet()
unet_model = load_model_weights_local_or_hf(
    unet_model,
    filename="results/xray_unet_epoch30.pth",
    repo_id="your-hf-username/xray-unet",
    subfolder="models"
)

unetpp_model = UNetPP()
unetpp_model = load_model_weights_local_or_hf(
    unetpp_model,
    filename="results/unetpp_epoch25.pth",
    repo_id="your-hf-username/mri-unetpp",
    subfolder="models"
)


# Utils
def preprocess(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    return transform(img).unsqueeze(0)

def overlay_prediction(img, mask):
    img = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img]*3, axis=-1)
    mask_bin = (mask > 0.5).astype(np.uint8)
    mask_rgb = np.zeros_like(img_rgb)
    mask_rgb[:, :, 1] = mask_bin * 255
    overlaid = (0.7 * img_rgb + 0.3 * mask_rgb).astype(np.uint8)
    return Image.fromarray(overlaid)


def predict_dcm(file):
    dcm = pydicom.dcmread(file.name)
    img = dcm.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    input_tensor = preprocess(img)
    with torch.no_grad():
        pred = torch.sigmoid(unet_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(img, pred)
    return result


def predict_nii(file, slice_index):
    nifti = nib.load(file.name)
    data = nifti.get_fdata()
    slice_img = data[:, :, slice_index]
    norm_img = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
    input_tensor = preprocess(norm_img)
    with torch.no_grad():
        pred = torch.sigmoid(unetpp_model(input_tensor)).squeeze().numpy()
    result = overlay_prediction(norm_img, pred)
    return result


def xray_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🩻 Chest X-ray Segmentation")
        file_input = gr.File(label="Upload DICOM (.dcm)")
        output_img = gr.Image(label="Overlayed Prediction")
        file_input.change(predict_dcm, inputs=file_input, outputs=output_img)
    return demo


def mri_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Brain MRI Segmentation")
        file_input = gr.File(label="Upload NIfTI (.nii.gz)")
        slice_slider = gr.Slider(0, 150, step=1, label="Slice Index", value=60)
        output_img = gr.Image(label="Overlayed Prediction")
        gr.Button("Predict").click(predict_nii, inputs=[file_input, slice_slider], outputs=output_img)
    return demo


def main():
    with gr.TabbedInterface([xray_interface(), mri_interface()], ["X-ray", "MRI"]):
        pass


if __name__ == '__main__':
    main()
